## Testing Edgar API

First test out the paid version, and then look at the free version afterwards

https://pypi.org/project/sec-api/#10-k10-q8-k-section-extractor-api

https://sec-api.io/pricing

Get the CIK key from the experiment - and try to do multiple calls later - to get the dataset needed

Maybe multiple calls with pagination - as seen before.

In [1]:
!pip install sec-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00


## Trying public version of the API

https://www.sec.gov/edgar/sec-api-documentation

In [2]:
API_KEY = ""

In [3]:
from sec_api import QueryApi

In [4]:

queryApi = QueryApi(api_key=API_KEY)#"YOUR_API_KEY")

In [ ]:
# https://sec-api.io/sandbox/search-by-multiple-ciks 

# use the structure stated in the link above for this

Query here uses the Lucene query syntax

https://sec-api.io/resources/lucene-query-syntax-overview

Search also allows you to return 10,000 queries in one go - so that is enough and can do this with pagination, as seen in the link below:

https://sec-api.io/docs/query-api#request-parameters

In [8]:
# now query the details - to see if you can find the documents

In [17]:
# query = {
#   "query": "ticker:TSLA AND filedAt:[2020-01-01 TO 2020-12-31] AND formType:\"10-K\"",
#   "from": "0",
#   "size": "10",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# use relevant query with own code struct to get the items you need.

# queries need to be done with pagination - see what would be the links for pagination later.

query = {
    "query": "cik:1050122 AND formType:\"10-K\"",
    # "query": "cik:1176334 AND formType:\"10-K\"",
    # "query": "cik:(1176334 OR 1413898 OR 99780) AND formType:\"10-K\"",
    # "query": "cik:(0001050122) AND formType:\"10-K\"",
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

Does work - but the ids need to be set with the leading 0s removed!!

In [18]:
filings

{'total': {'value': 9, 'relation': 'eq'},
 'query': {'from': 0, 'size': 20},
 'filings': [{'id': '974f7848acb9cd4e9fc46d49b65b7c93',
   'accessionNo': '0001104659-07-019474',
   'cik': '1050122',
   'ticker': 'CTAC',
   'companyName': '1 800 CONTACTS INC',
   'companyNameLong': '1 800 CONTACTS INC (Filer)',
   'formType': '10-K',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'filedAt': '2007-03-15T13:54:42-04:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1050122/000110465907019474/0001104659-07-019474.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1050122/0001104659-07-019474-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1050122/000110465907019474/a07-5825_110k.htm',
   'entities': [{'companyName': '1 800 CONTACTS INC (Filer)',
     'cik': '0001050122',
     'irsNo': '870571643',
     'stateOfIncorporation': 'DE',
     'fiscalYearEnd': '1231',
     'type':

In [20]:
filings["filings"][0]

{'id': '974f7848acb9cd4e9fc46d49b65b7c93',
 'accessionNo': '0001104659-07-019474',
 'cik': '1050122',
 'ticker': 'CTAC',
 'companyName': '1 800 CONTACTS INC',
 'companyNameLong': '1 800 CONTACTS INC (Filer)',
 'formType': '10-K',
 'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
 'filedAt': '2007-03-15T13:54:42-04:00',
 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1050122/000110465907019474/0001104659-07-019474.txt',
 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1050122/0001104659-07-019474-index.htm',
 'linkToXbrl': '',
 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1050122/000110465907019474/a07-5825_110k.htm',
 'entities': [{'companyName': '1 800 CONTACTS INC (Filer)',
   'cik': '0001050122',
   'irsNo': '870571643',
   'stateOfIncorporation': 'DE',
   'fiscalYearEnd': '1231',
   'type': '10-K',
   'act': '34',
   'fileNo': '000-23633',
   'filmNo': '07696033',
   'sic': '5961 Retail-Catalog &amp; Mail-Order Ho

In [21]:
report_url = filings["filings"][0]["linkToFilingDetails"] # here for 10K but it would for other types of reports

Take the document URL and use that  in the extractor API for the right documents, for the specific section as required

Use the link under `linkToFilingDetails` for each of the executions above. 

In [5]:
from sec_api import ExtractorApi

extractorApi = ExtractorApi(API_KEY)

In [22]:
section_text = extractorApi.get_section(report_url, "1A", "text")

In [23]:
section_text

' Item 1A. Risk Factors \n\nPlease consider carefully the following risk factors and all other information contained in this report. The risks and uncertainties described below are risks that we currently believe to be material, but they are not the only risks that we face. Additional risks and uncertainties not presently known to us or that we currently believe are immaterial may also impair our business operations. Any of the following risks could harm our business, operating results and financial condition. This report contains forward-looking statements that involve known and unknown risks and uncertainties. These statements relate to our plans, objectives, expectations and intentions. Our actual results could differ materially from those discussed in these statements. Factors that could contribute to these differences include those discussed below and elsewhere in this report. \n\nRisks Relating to Our Business \n\nClearLab continues to incur substantial losses and requires ongoin

In [24]:
# just make sure to not iterate over all of them

# or just filter by specific section of web to get what you need

We can see example to get everything we need

https://sec-api.io/docs/sec-filings-item-extraction-api/python-example

Although we need to iterate

# otherwise need to scrape it

https://gist.github.com/anshoomehra/ead8925ea291e233a5aa2dcaa2dc61b2

Another workflow here

https://medium.com/@jan_5421/how-to-download-and-scrape-10-k-filings-from-sec-edgar-b0d245fc8d48

Here also shows that you need to make multiple API calls - which is not strange - so do that here!!

## API limits

There are no limits on the number of calls here

Just that there is a API traffic limit that is it.

https://sec-api.io/pricing